In [41]:
# for some basic operations
import numpy as np 
import pandas as pd 

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# for providing path
import os
from datetime import datetime
import re

CSV_SEP = ','

In [42]:
# test code block for regex
# 54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/xxx HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"
test = '54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/xxx HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"'
test = '5.211.97.39 - - [22/Jan/2019:03:57:16 +0330] "GET /image/64291/productModel/200x200 HTTP/1.1" 200 5735 "https://www.zanbil.ir/m/filter/p25%2Cb103%2Cb50" "Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_2 like Mac OS X) AppleWebKit/603.2.4 (KHTML, like Gecko) Version/10.0 Mobile/14F89 Safari/602.1" "-"'
test = '66.249.66.194 - - [22/Jan/2019:03:57:15 +0330] "GET /m/product/25958/59518/%D8%A7%D8%AA%D9%88-%D8%A8%D8%AE%D8%A7%D8%B1-%D9%BE%D8%A7%D8%B1%D8%B3-%D8%AE%D8%B2%D8%B1-%D9%85%D8%AF%D9%84-SI-602 HTTP/1.1" 200 21178 "-" "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.96 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" "-"'
source_pattern = re.compile('[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+')
datetime_pattern = re.compile('(\[[0-9]+/(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))/[0-9]{4}:[0-9]{2}:[0-9]{2}:[0-9]{2} \+[0-9]{4}\]')
m_http_pattern = re.compile('(GET|POST|DELETE|PUT) /m/[^/]*') # HTTP/[(0-9|\.)]*"')
http_pattern = re.compile('(GET|POST|DELETE|PUT) /[^/]*')

source_match = source_pattern.search(test)
datetime_match = datetime_pattern.search(test)
m_http_match = m_http_pattern.search(test)
if m_http_match == None:
    http_match = http_pattern.search(test)
else:
    http_match = m_http_match
    
print(source_match.group(0))
print(datetime_match.group(0))
print(http_match)


66.249.66.194
[22/Jan/2019:03:57:15 +0330]
<re.Match object; span=(48, 62), match='GET /m/product'>


In [43]:
month_translator = {
    "Jan": "01",
    "Feb": "02",
    "Mar": "03",
    "Apr": "04",
    "May": "05",
    "Jun": "06",
    "Jul": "07",
    "Aug": "08",
    "Sep": "09",
    "Oct": "10",
    "Nov": "11",
    "Dec": "12"
}

def format_datetime(raw_dt):
    _dt = raw_dt[1:-7]
    dt = f"{_dt[7:11]}-{month_translator[_dt[3:6]]}-{_dt[0:2]} {_dt[12:]}"
    datetime_obj = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
    
    return datetime_obj.timestamp()*1000 # convert to millis

my_http_pattern = re.compile("/[\S]*")
def format_http(raw_http):
    http = my_http_pattern.search(raw_http)
    if http == None:
        return raw_http
        
    http = http.group(0)
    if http[:3] == "/m/":
        http = http[2:]
        
    idx = http.find('?')
    if idx != -1:
        http = http[:idx]
    return http
    
print(format_datetime("[22/Jan/2019:03:56:14 +0330]"))
print(format_http("GET /m/filter HTTP/1.1"))
# print(format_http('"GET /filter HTTP/1.1"'))

1548100574000.0
/filter


In [44]:
cnt = 0
with open('access_log.csv', 'w+') as csv:
    csv.write(f"source{CSV_SEP}datetime{CSV_SEP}http\n")
    with open('./access.log', 'r') as raw:
        try:
            while(True):
                raw_line = next(raw)
                source_match = source_pattern.search(raw_line)
                datetime_match = datetime_pattern.search(raw_line)
                m_http_match = m_http_pattern.search(raw_line)
                if m_http_match == None:
                    http_match = http_pattern.search(raw_line)
                else:
                    http_match = m_http_match
                
                if source_match == None or datetime_match == None or http_match == None:
                    #print(f"[Error] Fail to parse line: {raw_line}")
                    continue
                source = source_match.group(0)
                dt = format_datetime(datetime_match.group(0))
                http = format_http(http_match.group(0))
                
                if CSV_SEP in http:
                    continue
                cnt+=1
                csv.write(f"{source}{CSV_SEP}{dt}{CSV_SEP}{http}\n")
                
                
        except StopIteration:
            print("[Info] EOF")


[Info] EOF


In [45]:
data = pd.read_csv('./access_log.csv', sep=CSV_SEP)
data.head()

,source,datetime,http
0,54.36.149.41,1.548101e+12,/filter
1,31.56.96.51,1.548101e+12,/image
2,31.56.96.51,1.548101e+12,/image
3,40.77.167.129,1.548101e+12,/image
4,91.99.72.15,1.548101e+12,/product


In [46]:
print("Unique source:")
print(data.source.value_counts()[:20])

Unique source:
66.249.66.194      353483
66.249.66.91       314522
151.239.241.163     91913
66.249.66.92        88332
91.99.30.32         45648
104.222.32.91       42046
91.99.72.15         38694
91.99.47.57         38526
5.78.190.233        37090
195.181.168.181     27963
23.101.169.3        27800
195.181.168.164     24037
66.249.66.93        22146
17.58.102.43        21698
104.222.32.94       16696
5.160.157.20        13526
5.117.116.238       13401
5.160.221.38        12058
172.20.2.174        11303
40.77.167.156       10231
Name: source, dtype: int64


In [47]:
print("Unique http:")
print(data.http.value_counts()[:20])   


Unique http:
/image                               5682613
/static                              1998868
/filter                               648398
/product                              356779
/settings                             352037
/site                                 240410
/rapidGrails                          197113
/browse                               157888
/favicon.ico                          102822
/amp-helper-frame.html                 91958
/discountLabel                         50175
/                                      36658
/orderAdministration                   35993
/ajaxFilter                            35060
/plugins                               33205
/article                               31760
/search                                25286
/basket                                19740
/apple-touch-icon-precomposed.png      17874
/apple-touch-icon.png                  16823
Name: http, dtype: int64


In [48]:
https = data.http.value_counts().keys()
cnt = 1
http_to_vnf = dict()
for value in https:
    http_to_vnf[value] = f"/rank{cnt}"
    cnt += 1
    
def map_http_to_vnf(http):
    return http_to_vnf.get(http, http)
    
ips = data.source.value_counts().keys()
cnt = 1
ip_to_node = dict()
for value in ips:
    ip_to_node[value] = f"node{cnt}"
    cnt += 1
    
def map_ip_to_node(ip):
    return ip_to_node.get(ip, ip)

print(map_http_to_vnf("/image"))
print(map_ip_to_node("66.249.66.194"))

/rank1
node1


In [49]:
with open('access_log.csv', 'w+') as csv:
    csv.write(f"source{CSV_SEP}datetime{CSV_SEP}http\n")
    
    for index, row in data.iterrows():
        source = map_ip_to_node(row["source"])
        http = map_http_to_vnf(row["http"])
        dt = row["datetime"]
        csv.write(f"{source}{CSV_SEP}{dt}{CSV_SEP}{http}\n")


In [50]:
data = pd.read_csv('./access_log.csv', sep=CSV_SEP)
data.head()

,source,datetime,http
0,node12276,1.548101e+12,/rank3
1,node29099,1.548101e+12,/rank1
2,node29099,1.548101e+12,/rank1
3,node2371,1.548101e+12,/rank1
4,node7,1.548101e+12,/rank4


vnf_1
node_1


In [51]:
print("Unique time:")
print(data.datetime.value_counts()[:1])

Unique time:
2019-01-26 19:07:39    368
Name: datetime, dtype: int64
